## Introduction & Dataset Overview

In this assignment, we work with a large-scale Reddit submissions dataset containing over 130,000 posts.

Each row represents a single Reddit submission, with metadata such as:

title – the text of the post

subreddit – the community it was posted in

number_of_upvotes / number_of_downvotes – voting metrics

score – Reddit’s official score (upvotes − downvotes)

total_votes – sum of upvotes + downvotes

rawtime / unixtime / localtime – posting timestamps

username – poster identity (sparsely available)

The goal of this project is to build a predictive model that can estimate a post’s popularity (score) using interpretable features derived from metadata and text.
Before performing any modeling, we begin with a thorough validation of data quality, ensuring that all fields behave according to Reddit’s definitions.

In [ ]:
import pandas as pd
import numpy as np
import html
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv(
    "/Users/benjaminma/Desktop/MGTA461+Data/CSE158project/AndrewFile/redditSubmissions.csv.gz",
    engine="python",
    on_bad_lines="skip",
)
df.head()

,#image_id,unixtime,rawtime,title,total_votes,reddit_id,number_of_upvotes,subreddit,number_of_downvotes,localtime,score,number_of_comments,username
0,0,1.333172e+09,2012-03-31T12:40:39.590113-07:00,And here's a downvote.,63470.0,rmqjs,32657.0,funny,30813.0,1.333198e+09,1844.0,622.0,Animates_Everything
1,0,1.333178e+09,2012-03-31T14:16:01.093638-07:00,Expectation,35.0,rmun4,29.0,GifSound,6.0,1.333203e+09,23.0,3.0,Gangsta_Raper
2,0,1.333200e+09,2012-03-31T20:18:33.192906-07:00,Downvote,41.0,rna86,32.0,GifSound,9.0,1.333225e+09,23.0,0.0,Gangsta_Raper
3,0,1.333252e+09,2012-04-01T10:52:10-07:00,Every time I downvote something,10.0,ro7e4,6.0,GifSound,4.0,1.333278e+09,2.0,0.0,Gangsta_Raper
4,0,1.333273e+09,2012-04-01T16:35:54.393381-07:00,Downvote &quot;Dies Irae&quot;,65.0,rooof,57.0,GifSound,8.0,1.333298e+09,49.0,0.0,Gangsta_Raper


### 🔍 Step 1 — Data Validation & Integrity Checks

Before running any exploratory analysis or building models, it is crucial to verify that the dataset is internally consistent and free of obvious structural issues.

We focus on four core checks:

#### 1. Score Consistency

Reddit defines:

score = upvotes − downvotes

We verify that this relationship holds for all posts.

#### 2. Non-Negative Vote Counts

Upvotes and downvotes are counts that must be zero or positive.
Any negative values would indicate corrupted rows.

#### 3. Duplicate Posts

The column reddit_id uniquely identifies each submission.
Duplicates would indicate reposted or corrupted data.

#### 4. Missing Values

We inspect which fields contain missing values and assess whether they affect modeling.
Notably, the username column is highly sparse, but all modeling-relevant columns are essentially complete.

These checks ensure the dataset is structurally sound before we move on to EDA and engineered features.

# BEN'S EDA IS RIGHT HERE - CAN REMOVE THIS LATER

In [ ]:
## however many cells needed. Note the structure of the data.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
eda_df = pd.read_csv("BenEDA/reddit_prepared.csv")
eda_df["post_date"] = pd.to_datetime(eda_df["post_date"])

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (15, 10)

# Create subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle("Reddit Post Analysis Dashboard", fontsize=16, fontweight="bold", y=1.00)

# 1. Best Hours to Post
ax1 = axes[0, 0]
hour_scores = eda_df.groupby("hour")["score"].mean().sort_index()
bars = ax1.bar(hour_scores.index, hour_scores.values, color="steelblue", alpha=0.7)
max_hour = int(hour_scores.idxmax())
bars[max_hour].set_color("red")
bars[max_hour].set_alpha(1.0)
ax1.axvline(x=17, color="red", linestyle="--", alpha=0.3, linewidth=2)
ax1.axvline(x=2, color="orange", linestyle="--", alpha=0.3, linewidth=2)
ax1.set_xlabel("Hour of Day")
ax1.set_ylabel("Average Score")
ax1.set_title(f"Best Time to Post: {max_hour}:00 (5pm=17, 2am=2)")
ax1.grid(axis="y", alpha=0.3)

# 2. Day of Week Pattern
ax2 = axes[0, 1]
day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
dow_scores = eda_df.groupby("day_of_week")["score"].mean()
colors = ["steelblue"] * 5 + ["coral", "coral"]
ax2.bar(range(7), dow_scores.values, color=colors, alpha=0.7)
ax2.set_xticks(range(7))
ax2.set_xticklabels(day_names)
ax2.set_ylabel("Average Score")
ax2.set_title("Weekend vs Weekday")
ax2.grid(axis="y", alpha=0.3)

# 3. Top Subreddits
ax3 = axes[0, 2]
top_subs = eda_df["subreddit"].value_counts().head(10)
ax3.barh(range(10), top_subs.values, color="teal", alpha=0.7)
ax3.set_yticks(range(10))
ax3.set_yticklabels(top_subs.index, fontsize=9)
ax3.set_xlabel("Number of Posts")
ax3.set_title("Most Active Subreddits")
ax3.invert_yaxis()
ax3.grid(axis="x", alpha=0.3)

# 4. Title Length vs Score
ax4 = axes[1, 0]
sample = eda_df.sample(min(3000, len(eda_df)))
ax4.scatter(sample["title_length"], sample["score"], alpha=0.05, s=20, color="purple")
ax4.set_xlabel("Title Length (characters)")
ax4.set_ylabel("Score")
ax4.set_title("Title Length Impact")
ax4.set_ylim(0, eda_df["score"].quantile(0.95))
ax4.grid(alpha=0.3)

# Add trend line
z = np.polyfit(eda_df["title_length"].dropna(), eda_df["score"], 1)
p = np.poly1d(z)
x_trend = np.linspace(eda_df["title_length"].min(), eda_df["title_length"].max(), 100)
ax4.plot(x_trend, p(x_trend), "r--", alpha=0.8, linewidth=2)

# 5. Score Distribution
ax5 = axes[1, 1]
ax5.hist(eda_df["score"], bins=50, color="steelblue", alpha=0.7, edgecolor="black")
ax5.axvline(
    eda_df["score"].mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Mean: {eda_df['score'].mean():.0f}",
)
ax5.axvline(
    eda_df["score"].median(),
    color="orange",
    linestyle="--",
    linewidth=2,
    label=f"Median: {eda_df['score'].median():.0f}",
)
ax5.set_xlabel("Score")
ax5.set_ylabel("Frequency (log scale)")
ax5.set_title("Score Distribution")
ax5.set_xlim(0, eda_df["score"].quantile(0.95))
ax5.set_yscale("log")
ax5.legend()
ax5.grid(alpha=0.3)

# 6. Correlation Heatmap (Top Features)
ax6 = axes[1, 2]
key_features = [
    "score",
    "hour",
    "day_of_week",
    "title_length",
    "has_question_mark",
    "positive_words",
    "author_avg_score",
]
corr_matrix = eda_df[key_features].corr()
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=1,
    ax=ax6,
    cbar_kws={"shrink": 0.8},
)
ax6.set_title("Feature Correlations")

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print(f"- Best hour to post: {max_hour}:00")
print(f"- Best day: {day_names[int(dow_scores.idxmax())]}")
print(f"- Average score: {eda_df['score'].mean():.1f}")
print(f"- Median score: {eda_df['score'].median():.1f}")
print(f"- Title length correlation: {eda_df['title_length'].corr(eda_df['score']):.3f}")


In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from plotnine import *
import altair as alt

alt.data_transformers.disable_max_rows()

# ==========================================================
# LOAD DATA (Polars)
# ==========================================================
reddit_pl = pl.read_csv("BenEDA/reddit_prepared.csv").with_columns(
    pl.col("post_date").str.strptime(pl.Datetime, strict=False)
)

# Convert once to pandas for plotting
reddit_pd = reddit_pl.to_pandas()

# ==========================================================
# 1. Best Hours to Post
# ==========================================================
hour_scores = (
    reddit_pl.group_by("hour")
    .agg(pl.col("score").mean().alias("mean_score"))
    .sort("hour")
    .to_pandas()
)

best_hour = int(hour_scores.loc[hour_scores["mean_score"].idxmax(), "hour"])

p1 = (
    alt.Chart(hour_scores)
    .mark_bar()
    .encode(
        x=alt.X("hour:O", title="Hour of Day"),
        y=alt.Y("mean_score:Q", title="Average Score"),
        color=alt.condition(
            alt.datum.hour == best_hour, alt.value("red"), alt.value("steelblue")
        ),
        tooltip=["hour", "mean_score"],
    )
    .properties(title=f"Best Hour to Post (Red = {best_hour}:00)")
)

# ==========================================================
# 2. Day of Week Pattern
# ==========================================================
dow_scores = (
    reddit_pl.group_by("day_of_week")
    .agg(pl.col("score").mean().alias("mean_score"))
    .sort("day_of_week")
    .to_pandas()
)

dow_scores["weekday"] = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
dow_scores["is_weekend"] = dow_scores["weekday"].isin(["Sat", "Sun"])

p2 = (
    alt.Chart(dow_scores)
    .mark_bar()
    .encode(
        x="weekday:N",
        y="mean_score:Q",
        color=alt.Color("is_weekend:N", scale=alt.Scale(range=["steelblue", "coral"])),
        tooltip=["weekday", "mean_score"],
    )
    .properties(title="Weekend vs Weekday")
)

# ==========================================================
# 3. Top Subreddits
# ==========================================================
top_subs = (
    reddit_pl.group_by("subreddit")
    .count()
    .sort("count", descending=True)
    .head(10)
    .to_pandas()
)

p3 = (
    alt.Chart(top_subs)
    .mark_bar()
    .encode(
        x="count:Q", y=alt.Y("subreddit:N", sort="-x"), tooltip=["subreddit", "count"]
    )
    .properties(title="Top 10 Active Subreddits")
)

# ==========================================================
# 4. Title Length vs Score (Scatter + Trendline)
# ==========================================================
sample_df = reddit_pd.sample(n=min(3000, len(reddit_pd)))

p4_scatter = (
    alt.Chart(sample_df)
    .mark_point(opacity=0.2, color="purple")
    .encode(x="title_length:Q", y="score:Q", tooltip=["title_length", "score"])
)

p4_line = p4_scatter.transform_regression("title_length", "score").mark_line(
    color="red"
)

p4 = (p4_scatter + p4_line).properties(title="Title Length Impact on Score")

# ==========================================================
# 5. Score Distribution
# ==========================================================
mean_score = reddit_pd["score"].mean()
median_score = reddit_pd["score"].median()

p5 = (
    alt.Chart(reddit_pd)
    .mark_bar()
    .encode(
        x=alt.X("score:Q", bin=alt.Bin(maxbins=50)), y="count()", tooltip=["count()"]
    )
    .properties(
        title=f"Score Distribution (Mean={mean_score:.0f}, Median={median_score:.0f})"
    )
)

# ==========================================================
# 6. Correlation Heatmap
# ==========================================================
key_features = [
    "score",
    "hour",
    "day_of_week",
    "title_length",
    "has_question_mark",
    "positive_words",
    "author_avg_score",
]

corr = reddit_pd[key_features].corr().stack().reset_index()
corr.columns = ["var1", "var2", "value"]

p6 = (
    alt.Chart(corr)
    .mark_rect()
    .encode(
        x="var1:N",
        y="var2:N",
        color=alt.Color("value:Q", scale=alt.Scale(scheme="redblue")),
        tooltip=["var1", "var2", "value"],
    )
    .properties(title="Feature Correlations")
)

# ==========================================================
# FULL DASHBOARD
# ==========================================================
dashboard = ((p1 | p2 | p3) & (p4 | p5 | p6)).resolve_scale(color="independent")

dashboard


## 1. Data Validation

Before building any models, we first validate that the core numeric fields in the dataset are internally consistent and that there are no obvious data quality issues.

**Checks performed:**

1. **Score consistency**
   We verify that the Reddit `score` field matches the definition:
      `score` = `number_of_upvotes` - `number_of_downvotes`

2. **Non-negative votes**

   We confirm that `number_of_upvotes` and `number_of_downvotes` are never negative.  

3. **Duplicated posts**

   Using `reddit_id` as the unique identifier of a submission, we check for duplicated `reddit_id`s.  

4. **Missing values**

   Finally, we compute the total number of missing values in the dataset
      - The `username` column has 20,260 missing values, but will not be considered in our model
      - Every other column only has 1 missing value on the same row

In [ ]:
# Score = upvotes - downvotes
score_diff = (
    (df["number_of_upvotes"] - df["number_of_downvotes"] - df["score"]).abs().sum()
)
print("Score consistency check (should be 0):", score_diff)

# Non-negative votes

neg_votes = ((df["number_of_upvotes"] < 0) | (df["number_of_downvotes"] < 0)).sum()
print("Number of rows with negative votes (should be 0):", neg_votes)

dup_count = df["reddit_id"].duplicated().sum()
print("Number of duplicated reddit_id values:", dup_count)

# Missing Values
df_nan = df.isna().sum()
print(df_nan)


#### The validation checks show that the dataset is high-quality and suitable for modeling, with a few minor notes:

Score consistency holds perfectly for all rows
Every post satisfies the Reddit definition score = upvotes − downvotes.

No negative vote counts were found, confirming that the voting data is well-formed.

A small number of duplicated reddit_id values exist (93 duplicates)
This is less than 0.1% of the dataset and likely corresponds to reposts or duplicate entries.
These duplicates can be safely ignored or removed since they do not materially affect modeling.

Missing values are extremely limited -->
Each modeling-relevant column has only 1 missing value, all occurring in the same row.
The username field is heavily missing (20k+ rows) but is not used in prediction.

Overall, the dataset is clean and consistent.
We can confidently proceed to feature engineering and exploratory analysis.

# Data Cleaning

#### 2. Data Cleaning & Pre-Processing

Before performing exploratory analysis or modeling, we apply a series of data-cleaning steps to ensure that all fields are in a consistent format and ready for feature engineering.

Reddit data often includes HTML artifacts, inconsistent types, missing values, and duplicated metadata.
This section standardizes those fields.

#### Cleaning Steps Performed:
##### 1. Type Casting

Several fields (e.g., vote counts, timestamps) are loaded as strings.
To make them usable, we convert them into numerical and datetime formats:

* unixtime → numeric

* datetime → converted using Unix timestamps

* total_votes, number_of_upvotes, number_of_downvotes, score → cast to float

2. Duplicate Post Removal

We drop duplicate posts based on reddit_id, since reposts or duplicate lines may appear in the raw dataset.

3. Missing Value Cleanup

We remove username entirely (20k+ missing values, not used in modeling).

All remaining rows with missing data are dropped (only 1 row removed).

4. Title Text Normalization

Reddit titles frequently contain:

* HTML escape sequences (e.g., &quot;, &amp;)

* Extra whitespace

* Mixed casing

We clean titles by:

* Unescaping HTML

* Lowercasing

* Collapsing repeated whitespace

* Ensuring all titles are strings

This step is essential to prepare the text for TF-IDF vectorization used later in modeling.

In [ ]:
# Casting
df["unixtime"] = pd.to_numeric(df["unixtime"], errors="coerce")
df["datetime"] = pd.to_datetime(df["unixtime"], unit="s")
df["total_votes"] = df["total_votes"].astype(float)
df["number_of_upvotes"] = df["number_of_upvotes"].astype(float)
df["number_of_downvotes"] = df["number_of_downvotes"].astype(float)
df["score"] = df["score"].astype(float)

# Duplicate Handling
df = df.drop_duplicates(subset=["reddit_id"])
dup_reddit_ids = df["reddit_id"].duplicated().sum()
print("Number of duplicated reddit_id values:", dup_reddit_ids)

# Missing Values Handling
df = df.drop(columns=["username"])
df = df.dropna()
print("Number of NANs:", df.isna().sum().sum())


def clean_title(text):
    # 1. Ensure string
    s = str(text)
    # 2. Unescape HTML entities: &quot; &amp; &lt; &gt; etc.
    s = html.unescape(s)
    # 3. Strip leading/trailing whitespace
    s = s.strip()
    # 4. Collapse multiple spaces/newlines into a single space
    s = re.sub(r"\s+", " ", s)
    # 5. (Optional) lowercase for modeling
    s = s.lower()
    return s


df["title_clean"] = df["title"].apply(clean_title)
df["title"] = df["title_clean"]
df.drop(columns=["title_clean"], inplace=True)

### Summary of Cleaning Actions

#### After applying the cleaning pipeline:

* All numeric fields are correctly typed and ready for aggregation, modeling, and feature engineering.

* All duplicated reddit_id rows were removed, leaving a unique set of Reddit submissions.

* Only 1 row with missing values was dropped, resulting in a nearly complete dataset.

* Title text is normalized for NLP-based feature extraction (TF-IDF), improving model performance.

* The final dataset is clean, consistent, and prepared for the next stage: Exploratory Data Analysis (EDA).

### Handling Outliers in Score Distribution

Reddit post scores are extremely heavy-tailed. Even after applying a log-transform, we found that the top 1–5% of posts had massively disproportionate influence on our model.
These extreme values caused:

* Instability during model training

* Very large error spikes

* Poor generalization on typical posts

To address this, we remove the top 5% of scores. This is a standard approach when working with highly skewed social-media engagement data.

Removing these outliers makes the score distribution far more well-behaved and leads to significantly improved MAE in both log and raw score space.

In [ ]:
p95 = df["score"].quantile(0.95)
df = df[df["score"] <= p95].copy()


# this move massively improved our model

# I dont have a graph but i'm pretty sure the values towards the top are astronomically large

In [ ]:
print(df.shape)
df.head()

#### Effect of Outlier Removal

After filtering the top 5% of scores:

* The dataset becomes more statistically stable

* The log-transformed target behaves normally

* Extreme score values no longer dominate the model

* Downstream models produce much lower and more interpretable MAE

This step was crucial — without it, even strong models (e.g., boosted trees) struggled.
After outlier removal, model performance improved dramatically.

# Feature Engineering

To improve model performance, we engineered several new features capturing temporal patterns, linguistic structure, and semantic cues in the title text.
Below we outline each feature category and show the transformations step-by-step.

### Temporal Features Extracted from Timestamps

Reddit engagement is strongly influenced by when a post is made (hour, weekday, month).
We extract several time-based predictors from the datetime column.

In [ ]:
df["hour"] = df["datetime"].dt.hour
df["dayofweek"] = df["datetime"].dt.dayofweek
df["month"] = df["datetime"].dt.month
df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)


### Title Structure Features

These features capture simple but meaningful signals from title text, such as length, wordiness, and whether a question or exclamation is being asked.

We compute:

* Total character length

* Word count

* If title contains a ?

* If title contains a !

In [ ]:
df["title"] = df["title"].astype(str).str.strip()
df["title_length"] = df["title"].apply(len)
df["word_count"] = df["title"].apply(lambda x: len(x.split()))
df["is_question"] = df["title"].str.contains("?", regex=False).astype(int)
df["is_exclamation"] = df["title"].str.contains("!", regex=False).astype(int)


### Subreddit-Level Statistics (Omitted to Avoid Target Leakage)

We considered adding subreddit-level aggregate features (e.g., avg score per subreddit), but this directly leaks target information from the training set into the features.

Therefore, this block is intentionally commented out and not used.

In [ ]:
# subreddit_stats = df.groupby('subreddit')['score'].agg(
#     subreddit_mean_score='mean',
#     subreddit_post_count='count'
# ).reset_index()
# df = df.merge(subreddit_stats, on='subreddit', how='left')


### Sentiment Features Using VADER

Engagement on Reddit is often influenced not just by what a post says but how it is expressed.
Emotional or provocative wording can drive significantly higher voting activity. To capture this effect, we extract sentiment features from the post title using the VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiment analyzer.

VADER is specifically designed for short, informal, and emotion-heavy text making it an ideal fit for Reddit titles, which often include sarcasm, slang, exaggeration, and expressive punctuation.

For each title, VADER produces four interpretable sentiment metrics:

* Negative (neg) – proportion of the text conveying negative emotion.
Captures anger, frustration, complaints, or pessimistic tone.

* Neutral (neu) – proportion of the text that is objective, factual, or emotionally flat.
Neutral posts often receive lower engagement, making this a useful baseline signal.

* Positive (pos) – proportion of words carrying positive emotional tone.
Titles expressing excitement, humor, or appreciation often perform differently in voting dynamics.

* Compound (compound) – a normalized score from –1 to +1 summarizing overall sentiment.
This metric balances the intensity and direction of emotion and is one of the strongest indicators of emotional valence.

Adding these sentiment-derived features helps the model understand whether emotionally charged language (such as outrage, excitement, or humor) influences how many upvotes/downvotes a post receives.
This additional signal improves prediction accuracy, especially when combined with TF–IDF features in our Ridge regression model.

In [ ]:
sia = SentimentIntensityAnalyzer()


def add_title_sentiment(df):
    titles = df["title"].astype(str)
    scores = titles.apply(sia.polarity_scores)
    scores_df = scores.apply(pd.Series)

    df["title_sent_neg"] = scores_df["neg"]
    df["title_sent_neu"] = scores_df["neu"]
    df["title_sent_pos"] = scores_df["pos"]
    df["title_sent_compound"] = scores_df["compound"]
    return df


df = add_title_sentiment(df)


In [ ]:
df

# Modeling

Now that our dataset is fully cleaned and feature-engineered, we move into the modeling phase.

#### This section covers:

1. Defining the feature matrix (X) and target variable (y)

2. Applying log transformation to stabilize score values

3. Creating train/test splits

4. Building the final Ridge Regression + TF-IDF model

5. Evaluating performance in both log and original score space

#### Defining X and y

In this step, we select the feature columns created during feature engineering and assemble our input matrix X.

We also construct the target variable y using a stabilized log transformation:

y=log (score−min (score)+1)

This transformation reduces the influence of extremely large Reddit scores, making the model more stable and improving predictive performance.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# ----- 1. Define X and y -----

# Features
text_features = ["title"]
cat_features = ["subreddit"]
num_features = [
    "hour",
    "dayofweek",
    "month",
    "is_weekend",
    "title_length",
    "word_count",
    "is_question",
    "is_exclamation",
    "title_sent_neg",
    "title_sent_neu",
    "title_sent_pos",
    "title_sent_compound",
]

feature_cols = text_features + cat_features + num_features

X = df[feature_cols].copy()

# Target: use log(1 + score) to stabilize heavy tails
y_raw = df["score"]
min_score = y_raw.min()

# shift so that the minimum becomes 1, then log-transform
y_shifted = y_raw - min_score + 1
y_log = np.log(y_shifted)

print("X shape:", X.shape)
print("y shape:", y_log.shape)


The feature matrix and target vector are now prepared.
Next, we split the dataset into training and testing subsets to evaluate the model on unseen data.

### 2. Train/Test Split

We use an 80/20 split.
A fixed random_state ensures that results are reproducible.

In [ ]:
# ----- 2. Train / test split: 80 / 20 -----

RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=RANDOM_STATE
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


At this point, the data is fully ready for modeling.
Next, we construct the preprocessing pipeline (TF-IDF, One-Hot Encoding, scaling) and train our final Ridge Regression model.

### 3. Preprocessing + TF-IDF Vectorization

This block prepares all input features:

Text titles → TF-IDF

Categorical subreddit → One-Hot Encoding

Numerical features → Passthrough

This ensures that mixed data types can be fed directly into a model.

#### Baseline Models

Before training more sophisticated models, we establish baselines to understand how difficult the prediction problem is.
Baselines tell us whether our later models are actually learning anything meaningful.

We compute two baselines:

### Baseline 1: Predict Global Mean


The simplest possible prediction strategy is to ignore all features and always predict the global average training score.

Steps:

Compute the mean of the log-transformed scores on the training set

Predict this constant value for every test example

Inversely transform predictions back into raw score units

Compute MAE in both log space and raw score space

This gives us a lower-bound benchmark: any real model must beat this.

In [ ]:
from sklearn.metrics import mean_absolute_error

# 1. Baseline in log space
mean_pred = y_train.mean()
y_test_pred_log = np.full_like(y_test, mean_pred, dtype=float)

# 2. Inverse-transform both true and predicted values back to raw score
y_test_raw = np.exp(y_test) + min_score - 1
y_test_pred_raw = np.exp(y_test_pred_log) + min_score - 1

# 3. MAE in original score units (upvotes minus downvotes)
baseline_mae_raw = mean_absolute_error(y_test_raw, y_test_pred_raw)

print(f"Baseline MAE (log space): {mean_absolute_error(y_test, y_test_pred_log):.4f}")
print(f"Baseline MAE (raw score): {baseline_mae_raw:.4f}")


### Baseline 2 - Predict the Subreddit Mean (with Fallback)

This baseline takes a small step beyond the global average by using subreddit-level information.

The idea is straightforward:

Posts from the same subreddit often get similar amounts of attention.
So for each post in the test set, we predict its score using the average log-score of that subreddit in the training data.

If a subreddit never appeared in training, we simply fall back to the overall global mean so the model can still make a prediction.

Why this baseline is useful:

* Different subreddits naturally have different engagement levels

* The method is extremely simple to compute

* It gives us a stronger comparison point than the global mean alone

As with the first baseline, we report MAE in both log space and raw score space to keep our evaluation consistent.

For this baseline, we try something smarter than predicting the same score for every post.
Instead, we assume:

Posts from the same subreddit tend to receive similar levels of engagement.

So for every post, we predict the average log-score of its subreddit, computed only using the training set.

When the model encounters a subreddit in the test set that wasn't in training, we simply fall back to the global mean log-score.

This gives us a very interpretable baseline and helps us understand how much predictive power subreddit identity carries before applying full ML models.

#### 1. Compute Subreddit Mean Log-Scores (Train Only)

In this step, we calculate the average log-score for each subreddit using only the training split.
This prevents data leakage and keeps the baseline fair.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ---- Baseline 2: Subreddit mean (with fallback to global mean) ----

# 1. Compute mean *log-score* per subreddit on TRAIN ONLY
train_df_for_means = pd.DataFrame(
    {"subreddit": X_train["subreddit"].values, "score_log": y_train}
)  #  y_train is already log-transformed

subreddit_means_log = train_df_for_means.groupby("subreddit")["score_log"].mean()

# Global mean in log space (used as fallback for unseen subreddits)
global_mean_log = y_train.mean()

#### 2. Build a Simple Prediction Function

We create a helper that:

Looks up the subreddit’s mean log-score

Uses the global mean if that subreddit never appeared in training

In [ ]:
def predict_subreddit_mean_log(X, subreddit_means_log, global_mean_log):
    """
    For each row in X, predict the mean *log-score* of its subreddit,
    falling back to the global mean (log) if the subreddit was unseen in training.
    """
    return X["subreddit"].map(subreddit_means_log).fillna(global_mean_log).values


#### 3. Make Predictions on the Test Set

We run the baseline predictor on the test data in log-space:

In [ ]:
# 2. Predictions on TEST set (log space)
y_test_pred_log = predict_subreddit_mean_log(
    X_test, subreddit_means_log, global_mean_log
)

#### 4. Evaluate the Baseline (Log Space + Raw Score)

Reddit scores vary hugely, so we report error:

in log space (model training space)

and in raw score space (actual upvotes − downvotes), which is easier to interpret

In [ ]:
# 3. Metrics in LOG space
mae_log = mean_absolute_error(y_test, y_test_pred_log)

# 4. Inverse transform both true and predicted back to RAW score space
# recall: y_log = log(score - min_score + 1)
y_test_raw = np.exp(y_test) + min_score - 1
y_test_pred_raw = np.exp(y_test_pred_log) + min_score - 1

mae_raw = mean_absolute_error(y_test_raw, y_test_pred_raw)

print(f"  Test MAE  (log):  {mae_log:.4f}")
print(f"  Test MAE  (raw):  {mae_raw:.4f}")

## Linear Regression with TF-IDF + Metadata Features

#### Linear Regression with TF-IDF + Metadata Features

Now that we’ve established our baselines, we move on to our first real machine learning model.
This model combines:

* TF-IDF title embeddings

* One-hot encoded subreddit

* Numeric metadata features (time, comment count, sentiment, etc.)

This gives the model a much richer representation than the baselines.

We break the modeling process into 4 parts:

1. Selecting feature subsets

2. Building the preprocessing pipeline

3. Training with cross-validation

4. Evaluating on the test set

#### 1. Selecting Feature Subsets

Here, we extract the text, categorical, and numeric features that were engineered earlier.
We also create separate views of the training and test sets that match the expected input for our pipeline.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

RANDOM_STATE = 42  # for CV shuffling

lin_feature_cols = text_features + cat_features + num_features

X_train_lin = X_train[lin_feature_cols].copy()
X_test_lin = X_test[lin_feature_cols].copy()

text_col = "title"
cat_cols = cat_features
num_cols = num_features

#### 2. Building the Preprocessing Pipeline

We use a ColumnTransformer to apply three different preprocessing steps at once:

TF-IDF for the post title

One-hot encoding for subreddit

Standard scaling for numeric features

All of this is wrapped in a scikit-learn pipeline so preprocessing happens automatically during training and prediction.

In [ ]:
preprocessor_lin = ColumnTransformer(
    transformers=[
        (
            "text",
            TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=5),
            text_col,
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            cat_cols,
        ),
        (
            "num",
            StandardScaler(),
            num_cols,
        ),
    ]
)

lin_model = LinearRegression()

lin_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor_lin),
        ("model", lin_model),
    ]
)


### 5-Fold Cross-Validation on the Training Set

#### 3. Cross-Validation (5-Fold)

To check that our model generalizes well, we run 5-fold cross-validation on the training set.
This gives us a more reliable estimate of performance than a single train/test split.

In [ ]:
# 5 fold CV for consistency
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

cv_scores = cross_val_score(
    lin_pipeline,
    X_train_lin,
    y_train,
    cv=cv,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
)


### Final training + test evaluation

4. Final Training and Evaluation

We now train the model on the full training set and evaluate on the test set.

As before, we report MAE in both:

* Log space (model's actual prediction space)

* Raw score space (original Reddit score units, easier to interpret)

We invert the log-transform using the same shifting we did earlier.

In [ ]:
lin_pipeline.fit(X_train_lin, y_train)

# Predictions in log space
y_test_pred_log_lin = lin_pipeline.predict(X_test_lin)

# Log-space metrics
lin_mae_log = mean_absolute_error(y_test, y_test_pred_log_lin)

# Inverse-transform back to raw score space
y_test_raw = np.exp(y_test) + min_score - 1
y_test_pred_raw_lin = np.exp(y_test_pred_log_lin) + min_score - 1

lin_mae_raw = mean_absolute_error(y_test_raw, y_test_pred_raw_lin)

print(f"  Test MAE  (log): {lin_mae_log:.4f}")
print(f"  Test MAE  (raw): {lin_mae_raw:.4f}")


### Champion Model: Ridge Regression with TF-IDF + Metadata Features

After testing multiple approaches, Ridge Regression turned out to be the strongest model.
It performs extremely well with high-dimensional TF-IDF text features and avoids overfitting through L2 regularization.

This model combines:

* TF-IDF representations of post titles

* One-hot encoded subreddit

* Scaled numeric metadata features

* Ridge Regression, tuned via cross-validation

We break the setup into 4 steps:

1. Preparing the training and test feature matrices

2. Building the preprocessing pipeline

3. Running a hyperparameter search (alpha)

4. Evaluating the final model (log space + raw score space)

### 1. Preparing Training and Test Data

We start by selecting the text, categorical, and numeric feature groups.
These were all created earlier during feature engineering.

We then define X_train_ridge and X_test_ridge to match the exact input expected by our Ridge model.

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

ridge_feature_cols = text_features + cat_features + num_features

X_train_ridge = X_train[ridge_feature_cols].copy()
X_test_ridge = X_test[ridge_feature_cols].copy()


text_col = "title"
cat_cols = cat_features
num_cols = num_features

### (2) Build the Preprocessing Pipeline


2. Building the Preprocessing Pipeline

Ridge Regression must receive numerical inputs, so we use a ColumnTransformer to process each feature type:

* TF-IDF for the title

* One-hot encoding for subreddit

* Scaling for numeric metadata

This ensures consistent preprocessing during both training and prediction.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        (
            "text",
            TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=5),
            text_col,
        ),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            cat_cols,
        ),
        (
            "num",
            StandardScaler(),
            num_cols,
        ),
    ]
)


### (3) Set Up Ridge + Grid Search

3. Hyperparameter Tuning (Grid Search)

We wrap Ridge Regression in a scikit-learn Pipeline and tune its key parameter alpha
using 5-fold cross-validation.

alpha controls how strong the L2 penalty is — higher = more regularization.

The model is evaluated using MAE in log-space, the same space we trained in.

In [ ]:
# 5 fold CV
base_ridge = Ridge(random_state=RANDOM_STATE)

ridge_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", base_ridge),
    ]
)

param_grid = {"model__alpha": [0.01, 0.1, 1, 3, 10, 30, 100]}

grid = GridSearchCV(
    estimator=ridge_pipeline,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",  #!!!! MAE in LOG space
    cv=5,
    n_jobs=-1,
    verbose=1,
)

# y_train is log-transformed already!!!!! dont forget
grid.fit(X_train_ridge, y_train)


best_ridge = grid.best_estimator_

### (4) Evaluate the Champion Ridge Model

#### 4. Final Evaluation (Log Space + Raw Score Space)

We evaluate the best Ridge model from the grid search.

We compute:

* MAE in log space → model’s training space

* MAE in raw space → the actual Reddit score units

Raw MAE is the most interpretable metric, so we treat that as our final benchmark.

In [ ]:
# Predict in log(shifted score) space
y_test_pred_log = best_ridge.predict(X_test_ridge)
# Metrics in LOG space
mae_log = mean_absolute_error(y_test, y_test_pred_log)

y_test_pred_log = best_ridge.predict(X_test_ridge)
# log metrics
mae_log = mean_absolute_error(y_test, y_test_pred_log)
# back to raw
y_test_raw = np.exp(y_test) + min_score - 1
y_test_pred_raw = np.exp(y_test_pred_log) + min_score - 1
mae_raw = mean_absolute_error(y_test_raw, y_test_pred_raw)
print("Ridge MAE (log):", mae_log)
print("Ridge MAE (raw):", mae_raw)

### Why I Chose Ridge Regression as the Final Model

After experimenting with multiple modeling approaches for predicting Reddit post scores, I ultimately selected Ridge Regression as my final model. This decision wasn’t just based on theoretical expectations — it was strongly supported by the structure of my dataset and the performance patterns I saw in my experiments.

#### 1. Ridge fits the structure of my TF-IDF + metadata feature space

Once I applied TF-IDF to the post titles (with up to 20,000 features and bi-grams), my dataset became extremely high-dimensional and sparse. This is exactly the type of setting where Ridge Regression is known to work well.

Here’s what I found when comparing it to other models:

* Tree-based models like Random Forest and Gradient Boosting performed very poorly.
In my tests, they produced MAE values in the hundreds, meaning they failed to generalize on sparse, high-dimensional inputs.

* Ordinary Linear Regression handled TF-IDF better than the trees, but it still showed clear signs of overfitting, landing around 42–43 MAE in raw score space.

Ridge Regression, on the other hand, immediately introduced stability.
Because it uses L2 regularization, it prevents the massive coefficient swings that tend to happen with thousands of correlated TF-IDF features. This behavior perfectly matches both the theory behind Ridge and the improvements I saw in practice.

#### 2. Ridge Regression clearly outperformed every other model I tested

The results made the decision very straightforward.

Across all of the models I tried — including:

* Global-mean and subreddit-mean baselines

* Linear Regression with TF-IDF

* Random Forest

* Gradient Boosting Regressor

* HistGradientBoosting

Stacking models

Ridge Regression with hyperparameter tuning

Ridge Regression consistently delivered the lowest MAE.

My best Ridge model achieved:

#### FINAL: 40.03 MAE in raw score space

This was a dramatic improvement:

#### Baselines: ~60–70 MAE

#### TF-IDF Linear Regression: ~42 MAE

#### Tree-based models: 100+ MAE

#### Stackers: Couldn’t outperform Ridge and often struggled with NaNs

The fact that Ridge cut the MAE almost in half compared to ordinary Linear Regression highlighted how important regularization was for this dataset.